# Implement the given NFA

In [2]:
def delta_nfa(state, char):
    d = {
        ('s','a'): {'s','t'},
        ('s','b'): {'s',},
        ('t','a'): {'u','s'},
        ('u','a'): {'v','s'},
    }
    
    return d.get((state,char), set())
    

Test by eye

In [3]:
delta_nfa('s','a')

{'s', 't'}

In [3]:
all_states = set('stuv')

In [4]:
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [5]:
# construct all subsets of states
alphabet = set('ab')
all_subsets = [frozenset(i) for i in powerset(all_states)]
all_subsets

[frozenset(),
 frozenset({'v'}),
 frozenset({'t'}),
 frozenset({'u'}),
 frozenset({'s'}),
 frozenset({'t', 'v'}),
 frozenset({'u', 'v'}),
 frozenset({'s', 'v'}),
 frozenset({'t', 'u'}),
 frozenset({'s', 't'}),
 frozenset({'s', 'u'}),
 frozenset({'t', 'u', 'v'}),
 frozenset({'s', 't', 'v'}),
 frozenset({'s', 'u', 'v'}),
 frozenset({'s', 't', 'u'}),
 frozenset({'s', 't', 'u', 'v'})]

In [6]:
# construct all possible transitions

intermediate_mapping = dict()
for subset in all_subsets:
    for char in alphabet:    
        next_state = set()
        for state in subset:
            ss = delta_nfa(state, char)
#             print(subset,state,char,ss)
            next_state.update(ss)
#             print(state,char,ss)
            intermediate_mapping.update({(subset,char):frozenset(next_state)})
intermediate_mapping

{(frozenset({'v'}), 'b'): frozenset(),
 (frozenset({'t', 'v'}), 'b'): frozenset(),
 (frozenset({'u'}), 'a'): frozenset({'s', 'v'}),
 (frozenset({'u'}), 'b'): frozenset(),
 (frozenset({'s', 'u'}), 'a'): frozenset({'s', 't', 'v'}),
 (frozenset({'v'}), 'a'): frozenset(),
 (frozenset({'u', 'v'}), 'a'): frozenset({'s', 'v'}),
 (frozenset({'t'}), 'a'): frozenset({'s', 'u'}),
 (frozenset({'t'}), 'b'): frozenset(),
 (frozenset({'t', 'u'}), 'a'): frozenset({'s', 'u', 'v'}),
 (frozenset({'t', 'u'}), 'b'): frozenset(),
 (frozenset({'s'}), 'a'): frozenset({'s', 't'}),
 (frozenset({'s', 'u'}), 'b'): frozenset({'s'}),
 (frozenset({'s', 't'}), 'a'): frozenset({'s', 't', 'u'}),
 (frozenset({'s', 't', 'u'}), 'a'): frozenset({'s', 't', 'u', 'v'}),
 (frozenset({'s', 't', 'u'}), 'b'): frozenset({'s'}),
 (frozenset({'s', 't', 'v'}), 'a'): frozenset({'s', 't', 'u'}),
 (frozenset({'t', 'u', 'v'}), 'b'): frozenset(),
 (frozenset({'s'}), 'b'): frozenset({'s'}),
 (frozenset({'s', 'v'}), 'b'): frozenset({'s'}),


In [7]:
# perform graph traversal to prune inaccessible states
start_state = 's'
# remember which states have been traversed
seen_states = set()
# perform traversal iteratively using a stack
states_stack = [{'s'}]
while len(states_stack) > 0:
#     pop off a node
    state = frozenset(states_stack.pop())
    seen_states.add(state)
    print('saw',state)
#     construct the set of children of this node using the delta function
    for char in alphabet:
        ns = set()
        for substate in state:
            child = delta_nfa(substate, char)
            ns.update(child)
        if ns not in seen_states:
            states_stack.append(ns)
# delete all ((state,char)=> states) pairs from the mapping
# where the state was not traversed
for key in list(intermediate_mapping.keys()):
    if key[0] not in seen_states :
        print('delete inaccessible state',set(key),'=>',set(intermediate_mapping[key]))
        del intermediate_mapping[key]

saw frozenset({'s'})
saw frozenset({'t', 's'})
saw frozenset({'t', 'u', 's'})
saw frozenset({'v', 't', 'u', 's'})
delete inaccessible state {'b', frozenset({'v', 't'})} => set()
delete inaccessible state {'b', frozenset({'v'})} => set()
delete inaccessible state {'a', frozenset({'u', 's'})} => {'v', 't', 's'}
delete inaccessible state {frozenset({'v', 'u'}), 'a'} => {'v', 's'}
delete inaccessible state {'b', frozenset({'t', 'u'})} => set()
delete inaccessible state {'b', frozenset({'u'})} => set()
delete inaccessible state {'a', frozenset({'t', 'u'})} => {'v', 'u', 's'}
delete inaccessible state {'a', frozenset({'v'})} => set()
delete inaccessible state {'b', frozenset({'u', 's'})} => {'s'}
delete inaccessible state {frozenset({'u'}), 'a'} => {'v', 's'}
delete inaccessible state {'a', frozenset({'v', 't', 's'})} => {'t', 'u', 's'}
delete inaccessible state {'b', frozenset({'v', 't', 'u'})} => set()
delete inaccessible state {'b', frozenset({'v', 'u', 's'})} => {'s'}
delete inaccessible

In [8]:
for i in intermediate_mapping.items():
    print("delta({}, '{}') = {}".format(set(i[0][0]),i[0][1],set(i[1])))

delta({'t', 'u', 's'}, 'b') = {'s'}
delta({'t', 'u', 's'}, 'a') = {'v', 't', 'u', 's'}
delta({'v', 't', 'u', 's'}, 'a') = {'v', 't', 'u', 's'}
delta({'s'}, 'b') = {'s'}
delta({'t', 's'}, 'b') = {'s'}
delta({'s'}, 'a') = {'t', 's'}
delta({'v', 't', 'u', 's'}, 'b') = {'s'}
delta({'t', 's'}, 'a') = {'t', 'u', 's'}
